In [65]:
from eventtrader.keys import SEC_API_KEY

In [ ]:
import asyncio
import websockets
import json
SERVER_URL = "wss://stream.sec-api.io"
WS_ENDPOINT = SERVER_URL + "?apiKey=" + SEC_API_KEY
async def websocket_client():
    try:
        async with websockets.connect(WS_ENDPOINT) as websocket:
            print("✅ Connected to:", SERVER_URL)
            while True:
                message = await websocket.recv()
                filings = json.loads(message)
                for f in filings:
                    print(f"Full filing info: {f}")
                    # print(f["accessionNo"], f["formType"], f["filedAt"], f["cik"])
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
# asyncio.run(websocket_client())
await websocket_client()

✅ Connected to: wss://stream.sec-api.io
Full filing info: {'id': 'fe54fc3acf57013c2989bbc9634a5403', 'accessionNo': '0001193125-25-026807', 'cik': '1359314', 'ticker': '', 'companyName': 'Northwestern Mutual Variable Life Account II', 'companyNameLong': 'Northwestern Mutual Variable Life Account II (Filer)', 'formType': '485APOS', 'description': 'Form 485APOS - Post-effective amendment [Rule 485(a)]', 'filedAt': '2025-02-14T10:25:02-05:00', 'linkToTxt': 'https://www.sec.gov/Archives/edgar/data/1359314/000119312525026807/0001193125-25-026807.txt', 'linkToHtml': 'https://www.sec.gov/Archives/edgar/data/1359314/000119312525026807/0001193125-25-026807-index.htm', 'linkToXbrl': '', 'linkToFilingDetails': 'https://www.sec.gov/Archives/edgar/data/1359314/000119312525026807/d919803d485apos.htm', 'entities': [{'companyName': 'Northwestern Mutual Variable Life Account II (Filer)', 'cik': '1359314', 'irsNo': '390509570', 'stateOfIncorporation': 'WI', 'fiscalYearEnd': '1231', 'type': '485APOS', 'a

# Filtering Rules

Original filing format: {'id': '2d18c83f53d9561f521252206a523cdb', 'accessionNo': '0001949846-25-000046', 'cik': '1773383', 'ticker': 'DT', 'companyName': 'Dynatrace, Inc.', 'companyNameLong': 'Dynatrace, Inc. (Subject)', 'formType': '144', 'description': 'Form 144 - Report of proposed sale of securities', 'filedAt': '2025-02-14T10:25:23-05:00', 'linkToTxt': 'https://www.sec.gov/Archives/edgar/data/1773383/000194984625000046/0001949846-25-000046.txt', 'linkToHtml': 'https://www.sec.gov/Archives/edgar/data/1773383/000194984625000046/0001949846-25-000046-index.htm', 'linkToXbrl': '', 'linkToFilingDetails': 'https://www.sec.gov/Archives/edgar/data/1773383/000194984625000046/xsl144X01/primary_doc.xml', 'entities': [{'companyName': 'Dynatrace, Inc. (Subject)', 'cik': '1773383', 'irsNo': '000000000', 'fiscalYearEnd': '0331', 'sic': '7372 Services-Prepackaged Software', 'undefined': '06 Technology)'}], 'documentFormatFiles': [{'sequence': '1', 'documentUrl': 'https://www.sec.gov/Archives/edgar/data/1773383/000194984625000046/xsl144X01/primary_doc.xml', 'type': '144', 'size': '\xa0'}, {'sequence': '1', 'documentUrl': 'https://www.sec.gov/Archives/edgar/data/1773383/000194984625000046/primary_doc.xml', 'type': '144', 'size': '2966'}, {'sequence': '\xa0', 'description': 'Complete submission text file', 'documentUrl': 'https://www.sec.gov/Archives/edgar/data/1773383/000194984625000046/0001949846-25-000046.txt', 'type': '\xa0', 'size': '4442'}], 'dataFiles': [], 'seriesAndClassesContractsInformation': []}


1. Filter filings based on the "formType":

- Keep filings where "formType" is one of ['8-K', '10-K', '10-Q', '8-K/A', '10-K/A', '10-Q/A']
- Discard any filing where "formType" does not match the above list


2. For each filing, process the data based on the "formType":
a. If "formType" is one of ['10-K', '10-Q', '10-K/A', '10-Q/A']:

- Check the 'dataFiles' array for an object where 'type' is 'XML'
- If found, extract the URL from the 'documentUrl' field within that object
- If no XML data is found, discard the filing


b. If "formType" is one of ['8-K', '8-K/A']:

- Check the 'dataFiles' array for an object where 'type' is 'XML'
- If found, extract the URL from the 'documentUrl' field within that object
- If not found, get the URL from the 'linkToTxt' field


3. Process the 'documentFormatFiles' array for each filing:

- Iterate through the 'documentFormatFiles' array
- For each object in the array, check if the 'type' field starts with 'EX-10.' or 'EX-99.'
- If the 'type' field matches 'EX-10.' or 'EX-99.', extract the URL from the 'documentUrl' field within that object
- Discard all other objects in the 'documentFormatFiles' array where the 'type' field does not start with 'EX-10.' or 'EX-99.'


4. Output a single line for each filing that includes:

- Base fields (e.g., "id", "companyName", "formType", "filingDate", etc.)
- Entity fields (e.g., "entityId", "entityName", etc.)
- A dictionary containing the extracted 'EX-10.x' and 'EX-99.x' exhibits along with their associated URLs


The goal is to have each row of the output contain all the relevant metadata for a single filing, including the primary form URL (based on the "formType" rules) and the associated 'EX-10.x' and 'EX-99.x' exhibit URLs (if any).

    RULE: Discard any filing where "formType" != ['8-K', '10-K', '10-Q', '8-K/A', '10-K/A', '10-Q/A']

    
    * Note ouput for each filing below will have a single line ouput with **all meta details for that filing**. Meaning each row will include base_fields, entity fields as well as a dictionary containing any 'EX-10.x' or 'EX-99.x' and their associated url (based on rules below)


    RULE IF "formType" == ['10-K', '10-Q', '10-K/A', '10-Q/A'], Keep following data:
        1. In 'dataFiles' > 'type' == 'XML', get url from 'documentUrl' (in 'dataFiles')
        2. Discard if no XML data is found

    RULE IF "formType" == ['8-K', '8-K/A'], Keep following data:
        1. If 'dataFiles' > 'type' == 'XML', get url from 'documentUrl' (in 'dataFiles') else get url from 'linkToTxt'

    RULE for finding EX-10.x and EX-99.x for each "formType" above:
        1. In 'documentFormatFiles' > 'type' == 'EX-10.x' or 'EX-99.x', get value from 'documentUrl' (in documentFormatFiles)
        2. Discard all other 'documentFormatFiles' > 'type' and in the same line store 'EX-10.x' or 'EX-99.x' along with their associated url ('documentUrl')

Overall idea is that each single line of output will contain primary forms such as ['8-K', '10-K', '10-Q', '8-K/A', '10-K/A', '10-Q/A'] along with their associated EX-10.x and EX-99.x exhibits urls.
        

### For storing the data in a Json file


In [ ]:
import asyncio
import websockets
import json
from eventtrader.keys import SEC_API_KEY


SERVER_URL = "wss://stream.sec-api.io"
WS_ENDPOINT = SERVER_URL + "?apiKey=" + SEC_API_KEY
OUTPUT_FILE = "sec_filings.json"  # File to save valid JSON data

async def websocket_client():
    try:
        async with websockets.connect(WS_ENDPOINT) as websocket:
            print("✅ Connected to:", SERVER_URL)
            filings_list = []

            while True:
                message = await websocket.recv()
                filings = json.loads(message)

                for f in filings:
                    filings_list.append(f)  # Append each valid JSON object

                # Save data every 10 filings
                if len(filings_list) % 1 == 0:
                    save_data(filings_list)

    except Exception as e:
        print(f"❌ An unexpected error occurred: {e}")

# Save the JSON data correctly
def save_data(filings_list):
    with open(OUTPUT_FILE, "w", encoding="utf-8") as f:
        json.dump(filings_list, f, indent=4)  # Ensure proper formatting
    print(f"✅ Data saved to {OUTPUT_FILE}")

# Run WebSocket client
await websocket_client()


✅ Connected to: wss://stream.sec-api.io
✅ Data saved to sec_filings.json
✅ Data saved to sec_filings.json
✅ Data saved to sec_filings.json
✅ Data saved to sec_filings.json
✅ Data saved to sec_filings.json
✅ Data saved to sec_filings.json
✅ Data saved to sec_filings.json
✅ Data saved to sec_filings.json
✅ Data saved to sec_filings.json
✅ Data saved to sec_filings.json
✅ Data saved to sec_filings.json
✅ Data saved to sec_filings.json
✅ Data saved to sec_filings.json
✅ Data saved to sec_filings.json
✅ Data saved to sec_filings.json
✅ Data saved to sec_filings.json
✅ Data saved to sec_filings.json
✅ Data saved to sec_filings.json
✅ Data saved to sec_filings.json
✅ Data saved to sec_filings.json
✅ Data saved to sec_filings.json
✅ Data saved to sec_filings.json
✅ Data saved to sec_filings.json
✅ Data saved to sec_filings.json
✅ Data saved to sec_filings.json
✅ Data saved to sec_filings.json
✅ Data saved to sec_filings.json
✅ Data saved to sec_filings.json
✅ Data saved to sec_filings.json
✅ D

#### Load saved JSON file

In [58]:
import json
import pandas as pd

# Load JSON file
file_path = "sec_filings.json"
with open(file_path, "r", encoding="utf-8") as file:
    filings_data = json.load(file)

# Flatten function to extract nested fields
def flatten_filing(filing):
    base_fields = {
        "id": filing.get("id"),
        "accessionNo": filing.get("accessionNo"),
        "cik": filing.get("cik"),
        "ticker": filing.get("ticker"),
        "companyName": filing.get("companyName"),
        "companyNameLong": filing.get("companyNameLong"),
        "formType": filing.get("formType"),
        "description": filing.get("description"),
        "filedAt": filing.get("filedAt"),
        "linkToTxt": filing.get("linkToTxt"),
        "linkToHtml": filing.get("linkToHtml"),
        "linkToXbrl": filing.get("linkToXbrl"),
        "linkToFilingDetails": filing.get("linkToFilingDetails"),
        "periodOfReport": filing.get("periodOfReport"),
        "effectivenessDate": filing.get("effectivenessDate"),
    }

    # Extract Entities
    entities = filing.get("entities", [])
    entity_rows = []
    for entity in entities:
        entity_row = base_fields.copy()
        entity_row.update({
            "entity_companyName": entity.get("companyName"),
            "entity_cik": entity.get("cik"),
            "entity_irsNo": entity.get("irsNo"),
            "entity_stateOfIncorporation": entity.get("stateOfIncorporation"),
            "entity_fiscalYearEnd": entity.get("fiscalYearEnd"),
            "entity_type": entity.get("type"),
            "entity_act": entity.get("act"),
            "entity_fileNo": entity.get("fileNo"),
            "entity_filmNo": entity.get("filmNo"),
            "entity_sic": entity.get("sic"),
        })
        entity_rows.append(entity_row)

    # Extract Documents
    documents = filing.get("documentFormatFiles", [])
    doc_rows = []
    for doc in documents:
        doc_row = base_fields.copy()
        doc_row.update({
            "doc_sequence": doc.get("sequence"),
            "doc_description": doc.get("description"),
            "doc_documentUrl": doc.get("documentUrl"),
            "doc_type": doc.get("type"),
            "doc_size": doc.get("size"),
        })
        doc_rows.append(doc_row)

    # Extract Data Files
    data_files = filing.get("dataFiles", [])
    data_rows = []
    for data in data_files:
        data_row = base_fields.copy()
        data_row.update({
            "data_sequence": data.get("sequence"),
            "data_description": data.get("description"),
            "data_documentUrl": data.get("documentUrl"),
            "data_type": data.get("type"),
            "data_size": data.get("size"),
        })
        data_rows.append(data_row)

    # Extract Series and Classes Contracts Information
    series_classes = filing.get("seriesAndClassesContractsInformation", [])
    series_rows = []
    for series in series_classes:
        series_row = base_fields.copy()
        series_row.update({
            "series_id": series.get("series"),
            "series_name": series.get("name"),
        })
        for contract in series.get("classesContracts", []):
            contract_row = series_row.copy()
            contract_row.update({
                "classContract": contract.get("classContract"),
                "classContract_name": contract.get("name"),
                "classContract_ticker": contract.get("ticker"),
            })
            series_rows.append(contract_row)

    return entity_rows + doc_rows + data_rows + series_rows

# Process all filings
all_filing_rows = []
for filing in filings_data:
    all_filing_rows.extend(flatten_filing(filing))

# Create DataFrame
df = pd.DataFrame(all_filing_rows)

df.head(2)

,id,accessionNo,cik,ticker,companyName,companyNameLong,formType,description,filedAt,linkToTxt,linkToHtml,linkToXbrl,linkToFilingDetails,periodOfReport,effectivenessDate,entity_companyName,entity_cik,entity_irsNo,entity_stateOfIncorporation,entity_fiscalYearEnd,entity_type,entity_act,entity_fileNo,entity_filmNo,entity_sic,doc_sequence,doc_description,doc_documentUrl,doc_type,doc_size,data_sequence,data_description,data_documentUrl,data_type,data_size,series_id,series_name,classContract,classContract_name,classContract_ticker
0,6a8edceafbcd082c49e7290de2591f72,0001213900-25-013179,1828098,STKH,Steakholder Foods Ltd.,Steakholder Foods Ltd. (Filer),6-K,Form 6-K - Report of foreign issuer [Rules 13a-16 and 15d-16],2025-02-13T08:07:15-05:00,https://www.sec.gov/Archives/edgar/data/1828098/000121390025013179/0001213900-25-013179.txt,https://www.sec.gov/Archives/edgar/data/1828098/000121390025013179/0001213900-25-013179-index.htm,,https://www.sec.gov/Archives/edgar/data/1828098/000121390025013179/ea0230980-6k_steakholder.htm,2025-02-13,None,Steakholder Foods Ltd. (Filer),1828098,000000000,L3,1231,6-K,34,001-40173,25617285,2000 Food and Kindred Products,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,6a8edceafbcd082c49e7290de2591f72,0001213900-25-013179,1828098,STKH,Steakholder Foods Ltd.,Steakholder Foods Ltd. (Filer),6-K,Form 6-K - Report of foreign issuer [Rules 13a-16 and 15d-16],2025-02-13T08:07:15-05:00,https://www.sec.gov/Archives/edgar/data/1828098/000121390025013179/0001213900-25-013179.txt,https://www.sec.gov/Archives/edgar/data/1828098/000121390025013179/0001213900-25-013179-index.htm,,https://www.sec.gov/Archives/edgar/data/1828098/000121390025013179/ea0230980-6k_steakholder.htm,2025-02-13,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,REPORT OF FOREIGN PRIVATE ISSUER,https://www.sec.gov/Archives/edgar/data/1828098/000121390025013179/ea0230980-6k_steakholder.htm,6-K,10988,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [59]:
# df.to_csv("sec_filings.csv", index=False)


### Data Types

In [60]:
from collections import Counter
# Get unique data types and their counts using Counter
data_type_counts = Counter(df.data_type.dropna())
data_type_counts


Counter({'EX-101.SCH': 35,
         'XML': 35,
         'EX-101.LAB': 33,
         'EX-101.PRE': 33,
         'EX-101.DEF': 17,
         'EX-101.CAL': 9})

In [61]:
df[df.data_type == "XML"]['data_documentUrl'].nunique()

35

In [62]:
Counter(df[df.data_type == "XML"]['formType'])

Counter({'8-K': 27, '10-K': 4, '10-Q': 2, 'N-CSRS': 1, 'S-1/A': 1})

In [21]:
Counter(df.formType)

Counter({'10-K': 575,
         '8-K': 301,
         '13F-HR': 114,
         '4': 97,
         '424B3': 46,
         '424B2': 45,
         '6-K': 29,
         '13F-NT': 28,
         'SCHEDULE 13G': 27,
         'S-1/A': 27,
         '10-Q': 26,
         'SCHEDULE 13G/A': 26,
         'N-CSRS': 23,
         'D': 20,
         '485BXT': 7,
         'FWP': 6,
         'D/A': 4,
         'DEFA14A': 3,
         'CERT': 3,
         '305B2': 3})

In [24]:
df.id.nunique(), df.accessionNo.nunique()

(120, 104)

In [25]:
df[df.data_type == "XML"].id.nunique()

35

In [27]:
Counter(df[(df.data_type == "XML") & (df.formType.isin(['8-K','10-K','10-Q']))].formType)

Counter({'8-K': 27, '10-K': 4, '10-Q': 2})

In [28]:
counter_result = df[(df.data_type == "XML") & (df.formType.isin(['8-K', '10-K', '10-Q']))].formType.value_counts()
counter_result


formType
8-K     27
10-K     4
10-Q     2
Name: count, dtype: int64

# Final Rules

#### Step 1: Filter for only these 3 report types - '8-K','10-K','10-Q' or for amendments: '8-K/A', '10-K/A', '10-Q/A'

In [2]:
form_types = ['8-K', '10-K', '10-Q']
reports_to_keep = form_types + [str(r)+'/A' for r in form_types]

# Since they are always in XML 
# mask1 = [df.formType.isin(reports_to_keep) & (df.data_type == "XML")]

reports_to_keep

['8-K', '10-K', '10-Q', '8-K/A', '10-K/A', '10-Q/A']

In [57]:
df[mask1]

KeyError: "None of [Index([(False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, ...)], dtype='object')] are in the [columns]"

In [43]:
print(f"Length of df before:{len(df)}")
# df = df[df.formType.isin(reports_to_keep) & (df.data_type == "XML")]
print(f"Length of df after:{len(df)}")

Length of df before:902
Length of df after:33


### Supplimentary documents to store (ALWAYS filed as part of a primary filing -  (10-K, 10-Q, or 8-K).)
    All EX-10.x (Material contracts)
    All EX-99.x (Miscellaneous exhibits)

In [31]:
df[df.formType.isin(['8-K', '10-K', '10-Q'])].doc_type.unique()

array([nan, '8-K', 'EX-99.1', 'GRAPHIC', '\xa0', '10-K', 'EX-3.1',
       'EX-4.1', 'EX-10.3C', 'EX-10.3D', 'EX-10.13', 'EX-10.15',
       'EX-10.16', 'EX-19', 'EX-21', 'EX-23', 'EX-31.1', 'EX-31.2',
       'EX-32', 'EX-99', 'EX-2.1', 'EX-99.2', 'EX-10.1', 'EX-10.2',
       'EX-10.3', 'EX-10.28', 'EX-10.29', 'EX-19.1', 'EX-21.1', 'EX-32.1',
       'EX-4.11', 'EX-10.07', 'EX-10.08', 'EX-10.09', 'EX-10.10',
       'EX-10.11', 'EX-10.17', 'EX-23.1', 'EX-32.2', '10-Q', 'EX-4.2',
       'EX-10.4'], dtype=object)

In [35]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)


In [49]:
# Filter for EX-10.x and EX-99.x
mask = df['doc_type'].str.startswith('EX-10.') | df['doc_type'].str.startswith('EX-99.')
# df[mask]


In [63]:
import pandas as pd
from collections import Counter

def filter_sec_filings(df):
    """
    Filter SEC filings:
    1. Keep 10-K/10-Q that have XBRL (checking all XBRL-related data_types)
    2. Keep all 8-K
    3. Keep their EX-10.x and EX-99.x exhibits
    """
    
    # Define XBRL-related patterns
    xbrl_pattern = r'XML|EX-101\.'
    
    # Get accession numbers of filings with XBRL data
    xbrl_accessions = df[
        df['data_type'].str.contains(xbrl_pattern, na=False)
    ]['accessionNo'].unique()
    
    # Print XBRL validation info
    print("\nXBRL Validation Summary:")
    ten_k_q = df[df['formType'].isin(['10-K', '10-Q'])]
    total_acc = ten_k_q['accessionNo'].unique()
    print(f"Total 10-K/10-Q accession numbers: {len(total_acc)}")
    print(f"With XBRL: {len(set(total_acc) & set(xbrl_accessions))}")
    print(f"Without XBRL: {len(set(total_acc) - set(xbrl_accessions))}")
    
    # Filter main documents
    main_forms = ['8-K', '10-K', '10-Q']
    form_pattern = '|'.join(f"^{form}(/A)?$" for form in main_forms)
    
    # For 10-K and 10-Q, must have XBRL
    main_docs = df[
        (df['formType'].str.match(form_pattern, na=False)) &
        (
            (df['formType'].str.startswith('8-K')) |  # Keep all 8-Ks
            (df['accessionNo'].isin(xbrl_accessions))  # Only 10-K/Q with XBRL
        )
    ].copy()
    
    # Get valid accession numbers
    valid_accessions = main_docs['accessionNo'].unique()
    
    # Get exhibits for valid accessions
    exhibit_pattern = r'^EX-(?:10\.|99\.)'
    exhibits = df[
        (df['accessionNo'].isin(valid_accessions)) &
        (df['doc_type'].str.match(exhibit_pattern, na=False))
    ].copy()
    
    # Combine and categorize
    filtered_df = pd.concat([main_docs, exhibits])
    
    # Add category
    def get_category(row):
        if pd.isna(row['doc_type']):
            return 'MAIN'
        elif row['doc_type'].startswith('EX-10.'):
            return 'MATERIAL_CONTRACT'
        elif row['doc_type'].startswith('EX-99.'):
            return 'MISC_EXHIBIT'
        return 'MAIN'  # Default to MAIN for non-exhibit docs
    
    filtered_df['document_category'] = filtered_df.apply(get_category, axis=1)
    
    # Additional validation info
    print("\nXBRL Data Types found:")
    print(df[df['data_type'].str.contains(xbrl_pattern, na=False)]['data_type'].value_counts())
    
    return filtered_df

if __name__ == "__main__":
    # Read CSV
    df = pd.read_csv('sec_filings.csv')
    
    # Apply filter
    filtered_df = filter_sec_filings(df)
    
    # Print summary statistics
    print("\nDocument Categories Summary:")
    print(filtered_df['document_category'].value_counts())
    
    print("\nForm Types Summary:")
    print(filtered_df[filtered_df['document_category'] == 'MAIN']['formType'].value_counts())
    
    print("\nExhibit Types Summary:")
    exhibits = filtered_df[filtered_df['document_category'].isin(['MATERIAL_CONTRACT', 'MISC_EXHIBIT'])]
    print(exhibits['doc_type'].value_counts())


XBRL Validation Summary:
Total 10-K/10-Q accession numbers: 6
With XBRL: 6
Without XBRL: 0

XBRL Data Types found:
data_type
EX-101.SCH    35
XML           35
EX-101.LAB    33
EX-101.PRE    33
EX-101.DEF    17
EX-101.CAL     9
Name: count, dtype: int64

Document Categories Summary:
document_category
MAIN                 857
MATERIAL_CONTRACT     46
MISC_EXHIBIT          44
Name: count, dtype: int64

Form Types Summary:
formType
10-K    561
8-K     270
10-Q     26
Name: count, dtype: int64

Exhibit Types Summary:
doc_type
EX-99.1     42
EX-10.1      6
EX-10.2      6
EX-10.15     4
EX-10.3      4
EX-10.3D     2
EX-10.3C     2
EX-99.2      2
EX-10.16     2
EX-10.13     2
EX-10.28     2
EX-10.29     2
EX-10.07     2
EX-10.08     2
EX-10.09     2
EX-10.10     2
EX-10.11     2
EX-10.17     2
EX-10.4      2
Name: count, dtype: int64


In [ ]:
df = pd.read_csv('your_sec_filings.csv')
filtered_df = filter_sec_filings(df)

In [ ]:
https://www.sec.gov/Archives/edgar/data/937098/000093709825000016/tnet-123124ex1028warren.htm